In [3]:
import numpy as np
import keras
from keras import backend as K
from keras.applications import MobileNet
from keras.applications.mobilenet import preprocess_input
from keras.models import Model
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.metrics import categorical_crossentropy
from keras.layers import Dense, Activation, Dropout, GlobalAveragePooling2D

In [5]:
CLASSES = 2 # missing pulley or perfect pulley

baseModel = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

x = baseModel.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
#x = Dense(1024, activation='relu')(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
preds = Dense(2, activation='softmax')(x)

In [6]:
model = Model(inputs=baseModel.input, outputs=preds)
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128 

In [8]:
for i, layer in enumerate(model.layers):
    print("{}: {}".format(i, layer))

0: <keras.engine.input_layer.InputLayer object at 0x7f22242a6d50>
1: <keras.layers.convolutional.ZeroPadding2D object at 0x7f22a8dcd6d0>
2: <keras.layers.convolutional.Conv2D object at 0x7f222423abd0>
3: <keras.layers.normalization.BatchNormalization object at 0x7f226c635e50>
4: <keras.layers.advanced_activations.ReLU object at 0x7f21eb8d0c10>
5: <keras.layers.convolutional.DepthwiseConv2D object at 0x7f222423acd0>
6: <keras.layers.normalization.BatchNormalization object at 0x7f222423a6d0>
7: <keras.layers.advanced_activations.ReLU object at 0x7f222418b290>
8: <keras.layers.convolutional.Conv2D object at 0x7f222418b7d0>
9: <keras.layers.normalization.BatchNormalization object at 0x7f21ebfedf10>
10: <keras.layers.advanced_activations.ReLU object at 0x7f21ebf86f90>
11: <keras.layers.convolutional.ZeroPadding2D object at 0x7f21ebf53690>
12: <keras.layers.convolutional.DepthwiseConv2D object at 0x7f21ebf415d0>
13: <keras.layers.normalization.BatchNormalization object at 0x7f21ebe86950>
14:

In [9]:
for layer in model.layers[:87]:
    layer.trainable = False

for layer in model.layers[87:]:
    layer.trainable = True

In [12]:
trainDataGen = ImageDataGenerator(preprocessing_function=preprocess_input, validation_split = 0.3)

trainGen = trainDataGen.flow_from_directory('/home/dongjun/djplace/samples/',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            #color_mode = 'rgb',
                                            class_mode = 'categorical',
                                            subset = 'training',
                                            shuffle = True)

Found 1823 images belonging to 2 classes.


In [13]:
validGen = trainDataGen.flow_from_directory('/home/dongjun/djplace/samples/',
                                           target_size = (224, 224),
                                           batch_size = 32,
                                           class_mode = 'categorical',
                                           subset = 'validation')

Found 780 images belonging to 2 classes.


In [15]:
FILEPATH = '/home/dongjun/djplace/samples/'
EPOCHS = 10
LEARNINGRATE = 0.001
DECAYRATE = LEARNINGRATE // EPOCHS
opt = Adam(lr = LEARNINGRATE, decay=DECAYRATE)

In [19]:
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [20]:
model.fit_generator(generator=trainGen,
                    steps_per_epoch = trainGen.n // trainGen.batch_size,
                    validation_data = validGen,
                    validation_steps = validGen.n // validGen.batch_size,
                    epochs = EPOCHS)

Epoch 1/10
56/56 [==============================] - 229s 4s/step - loss: 0.1873 - accuracy: 0.9352 - val_loss: 3.7582 - val_accuracy: 0.5312
Epoch 2/10
56/56 [==============================] - 241s 4s/step - loss: 0.1579 - accuracy: 0.9397 - val_loss: 2.6385 - val_accuracy: 0.5361
Epoch 3/10
56/56 [==============================] - 226s 4s/step - loss: 0.1367 - accuracy: 0.9475 - val_loss: 1.6243 - val_accuracy: 0.6150
Epoch 4/10
56/56 [==============================] - 228s 4s/step - loss: 0.1252 - accuracy: 0.9531 - val_loss: 2.2067 - val_accuracy: 0.6898
Epoch 5/10
56/56 [==============================] - 228s 4s/step - loss: 0.1194 - accuracy: 0.9492 - val_loss: 4.1510 - val_accuracy: 0.5642
Epoch 6/10
56/56 [==============================] - 228s 4s/step - loss: 0.1367 - accuracy: 0.9486 - val_loss: 2.1162 - val_accuracy: 0.6136
Epoch 7/10
56/56 [==============================] - 225s 4s/step - loss: 0.1162 - accuracy: 0.9520 - val_loss: 1.6732 - val_accuracy: 0.6364
Epoch 8/10
56